In [1]:
import cv2
import pandas as pd
import numpy as np
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torch.nn import CrossEntropyLoss

import os
from PIL import Image
import matplotlib.pyplot as plt

# Dataset Base Path

In [2]:
base_dir = 'C:/images/OX_images/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# training dataset Path
train_o_dir = os.path.join(train_dir, 'O')
train_x_dir = os.path.join(train_dir, 'X')
train_n_dir = os.path.join(train_dir, 'None')

# validation dataset Path
validation_o_dir = os.path.join(validation_dir, 'O')
validation_x_dir = os.path.join(validation_dir, 'X')
validation_n_dir = os.path.join(validation_dir, 'None')

# test dataset Path
test_o_dir = os.path.join(test_dir, 'O')
test_x_dir = os.path.join(test_dir, 'X')
test_n_dir = os.path.join(test_dir, 'None')

# Image data Pre-processing
### Augmentation

In [3]:
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((150,150)),
    transforms.RandomRotation(25),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(150, scale=(0.8, 1.2), ratio=(0.75, 1.33)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

validation_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

test_transforms = validation_transforms

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(validation_dir, transform=validation_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [4]:
class_indices = train_dataset.class_to_idx
print(class_indices)

{'None': 0, 'O': 1, 'X': 2}


# Model

In [5]:
class OX_Model_CNN(nn.Module):
    def __init__(self):
        super(OX_Model_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.conv3 = nn.Conv2d(32, 32, 3, 1, 1)
        
        self.fc1 = nn.Linear(32*18*18, 512)
        self.fc2 = nn.Linear(512, 3)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32*18*18)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
model = OX_Model_CNN()
print(model)

OX_Model_CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=10368, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)


In [6]:
optimizer = RMSprop(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
    model.eval()
    validation_loss = 0.0
    accuracy = 0.0
    best_loss = 100.0
    
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            accuracy += (predicted == labels).float().mean().item()
            if validation_loss < best_loss:
                torch.save(model.state_dict(), 'OX_class_model.pth')
                best_loss = validation_loss
            
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, "
          f"Validation Loss: {validation_loss / len(validation_loader)}, "
          f"Validation Accuracy: {accuracy / len(validation_loader)}")

Epoch 1/30, Loss: 3.425785097810957, Validation Loss: 1.0047202533797215, Validation Accuracy: 0.40789473684210525
Epoch 2/30, Loss: 1.0159441696272955, Validation Loss: 0.9344194405957272, Validation Accuracy: 0.7192982466597306
Epoch 3/30, Loss: 0.7526435057322184, Validation Loss: 0.6389174094227584, Validation Accuracy: 0.7631578947368421
Epoch 4/30, Loss: 0.43152809143066406, Validation Loss: 0.47205568494054634, Validation Accuracy: 0.75
Epoch 5/30, Loss: 0.31170423660013413, Validation Loss: 0.42184159505134294, Validation Accuracy: 0.7894736842105263
Epoch 6/30, Loss: 0.21349244564771652, Validation Loss: 0.32956921800130085, Validation Accuracy: 0.9078947368421053
Epoch 7/30, Loss: 0.21012482254041565, Validation Loss: 0.29410632730823155, Validation Accuracy: 0.9342105263157895
Epoch 8/30, Loss: 0.13280315200487772, Validation Loss: 0.24599438724320216, Validation Accuracy: 0.9210526315789473
Epoch 9/30, Loss: 0.09622875725229581, Validation Loss: 0.15155152910337896, Validat

# Model Evaluate

In [59]:
def evaluate_model(loader):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_accuracy += (predicted == labels).float().mean().item()
            
    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    return avg_loss, avg_accuracy

train_loss, train_accuracy = evaluate_model(train_loader)
print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")

validation_loss, validation_accuracy = evaluate_model(validation_loader)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")
            

Train Loss: 0.0017295093624852599, Train Accuracy: 1.0
Validation Loss: 0.36141845083620133, Validation Accuracy: 0.8571428571428571


In [60]:
torch.save(model.state_dict(), 'OX_class_model.pth')